# 교차 검증과 그리드 서치

## 검증 세트

In [1]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine-date')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [2]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split

# train과 test 분리 (train: 0.8, test: 0.2)
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

# train data를 다시 분리 (train: 0.64, validation: 0.16, test: 0.2)
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)
print(sub_input.shape, val_input.shape, test_input.shape, data.shape)
# print(sub_input.shape[0] / data.shape[0], val_input.shape[0] / data.shape[0], test_input.shape[0] / data.shape[0])

(4157, 3) (1040, 3) (1300, 3) (6497, 3)


In [4]:
from sklearn.tree import DecisionTreeClassifier

# 하나의 validation set으로만 성능 확인
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


## 교차 검증

In [32]:
import numpy as np
from sklearn.model_selection import cross_validate

# train data에 대해 cross validate 진행
scores = cross_validate(dt, train_input, train_target)
print(np.mean(scores['test_score']))
print("----------------------------")
for k, v in scores.items():
    '''
    fit_time, score_time: train 수행 시간, validation 수행 시간
    test_score: validation 점수
    '''
    print(f"{k}: {v}")

0.8554925223957948
----------------------------
fit_time: [0.00726295 0.00626111 0.00646925 0.00559282 0.00514102]
score_time: [0.00061798 0.00065303 0.00046182 0.00042605 0.00043607]
test_score: [0.87019231 0.84615385 0.87680462 0.84889317 0.83541867]


In [33]:
from sklearn.model_selection import StratifiedKFold

# cross_validate()은 기본적으로 StratifiedKFold()를 cv로 사용
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.8554925223957948


In [34]:
# 새로운 cv를 사용해 학습
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8581873425226026


In [35]:
# 전체 train 데이터로 다시 학습해 모델의 성능 확인
dt = DecisionTreeClassifier(random_state=42)
dt.fit(train_input, train_target)

print(dt.score(test_input, test_target))

0.8584615384615385


## 하이퍼파라미터 튜닝

### 그리드 서치

In [37]:
from sklearn.model_selection import GridSearchCV

# 그리드 서치를 위한 범위 지정
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

# 그리드 서치 수행
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [38]:
# 가장 좋은 성능의 estimator 분석
dt = gs.best_estimator_
print(dt.score(train_input, train_target))
print("----------------------------")

# best_estimator에 해당하는 param 출력
print(gs.best_params_)

# 각 param의 cross validation 결과 출력
print(gs.cv_results_['mean_test_score'])

0.9615162593804117
----------------------------
{'min_impurity_decrease': 0.0001}
[0.86800067 0.86453617 0.86492226 0.86780891 0.86761605]


In [39]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [43]:
# 다양한 param을 구성하고 그리드 서치 수행 (9 * 15 * 10의 경우의 수를 모두 탐색)
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth':range(5,20,1),
          'min_samples_split':range(2,100,10)
          }

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [44]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731


### 랜덤 서치

In [45]:
from scipy.stats import uniform, randint

# discrete uniform distribution 생성 
rgen = randint(0, 10)

# 10개 샘플링
rgen.rvs(10)

array([7, 3, 1, 3, 5, 8, 6, 3, 6, 3])

In [46]:
# 샘플링을 1000개로 늘려 각 정수의 빈도 확인 
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([105,  91, 105,  99,  80, 112, 103,  96,  87, 122]))

In [47]:
# continuous uniform distribution 생성
ugen = uniform(0, 1)

# 10개 샘플링
ugen.rvs(10)

array([0.81485333, 0.24993009, 0.42738986, 0.20605636, 0.29473314,
       0.78410165, 0.91730252, 0.05601948, 0.33430967, 0.79730631])

In [48]:
from sklearn.model_selection import RandomizedSearchCV

params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25)}

# 랜덤 서치 수행 (무작위로 n_iter개의 경우의 수를 탐색)
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x15fd639d0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x15feefd60>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x15fd61060>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x15fd600d0>},
                   random_state=42)

In [49]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))
print("----------------------------")
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884
----------------------------
0.86


## 확인문제

In [50]:
# 분할 후보에서 무작위로 선택 ('best'의 경우 후보 중 최적의 분할을 선택)
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x15fd639d0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x15feefd60>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x15fd61060>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x15fd600d0>},
                   random_state=42)

In [51]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))
print("----------------------------")
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
----------------------------
0.786923076923077
